In [1]:
import base64
import datetime
import time
import json
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from pylab import rcParams
import seaborn as sns
import statsmodels as sm
from pylab import rcParams
from numpy import median
import matplotlib.dates as mdates

np.random.seed(1337)
%matplotlib inline
sns.set(font_scale=1.5)
rcParams['figure.figsize'] = 12, 8
sns.set_style('whitegrid')
sns.set_palette(sns.color_palette('muted'))

In [168]:
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier

## Load the Sample Data

In [2]:
d = pd.read_csv('tmp/clean_sample_10000.csv',
        parse_dates=[
        'measurement_date',
        'first_contract_date'])
len(d)

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2705: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


152088

## Aggregate by customer_id

In [3]:
dg = d.groupby(['customer_id'])
len(dg)

10000

In [4]:
zip(range(len(dg.head(1).head(1).columns)), dg.head(1).head(1).columns)

[(0, 'measurement_date'),
 (1, 'customer_id'),
 (2, 'employment_index'),
 (3, 'country_of_residence'),
 (4, 'gender'),
 (5, 'age'),
 (6, 'first_contract_date'),
 (7, 'is_new_customer'),
 (8, 'customer_for_months'),
 (9, 'is_primary_customer'),
 (10, 'last_primary_customer_date'),
 (11, 'customer_type'),
 (12, 'customer_relation_type'),
 (13, 'country_residence_bank_same'),
 (14, 'is_foreigner'),
 (15, 'is_spouse_of_an_employee'),
 (16, 'join_channel'),
 (17, 'is_dead'),
 (18, 'address_type'),
 (19, 'province_code'),
 (20, 'is_active_customer'),
 (21, 'household_income'),
 (22, 'customer_segment'),
 (23, 'ind_ahor_fin_ult1'),
 (24, 'ind_aval_fin_ult1'),
 (25, 'ind_cco_fin_ult1'),
 (26, 'ind_cder_fin_ult1'),
 (27, 'ind_cno_fin_ult1'),
 (28, 'ind_ctju_fin_ult1'),
 (29, 'ind_ctma_fin_ult1'),
 (30, 'ind_ctop_fin_ult1'),
 (31, 'ind_ctpp_fin_ult1'),
 (32, 'ind_deco_fin_ult1'),
 (33, 'ind_deme_fin_ult1'),
 (34, 'ind_dela_fin_ult1'),
 (35, 'ind_ecue_fin_ult1'),
 (36, 'ind_fond_fin_ult1'),
 (37,

In [5]:
g1 = dg.get_group(1054024).sort_values(by=['measurement_date'])
g1

,measurement_date,customer_id,employment_index,country_of_residence,gender,age,first_contract_date,is_new_customer,customer_for_months,is_primary_customer,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
60,2015-01-28,1054024,not-employee,ES,female,30.0,2012-08-27,no,35.0,yes,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
12939,2015-02-28,1054024,not-employee,ES,female,30.0,2012-08-27,no,35.0,yes,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
19245,2015-03-28,1054024,not-employee,ES,female,30.0,2012-08-27,no,35.0,yes,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
26051,2015-04-28,1054024,not-employee,ES,female,30.0,2012-08-27,no,35.0,yes,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
26245,2015-05-28,1054024,not-employee,ES,female,30.0,2012-08-27,no,35.0,yes,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
32847,2015-06-28,1054024,not-employee,ES,female,30.0,2012-08-27,no,35.0,yes,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
45592,2015-07-28,1054024,not-employee,ES,female,30.0,2012-08-27,no,35.0,yes,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
53519,2015-08-28,1054024,not-employee,ES,female,30.0,2012-08-27,no,36.0,yes,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
63005,2015-09-28,1054024,not-employee,ES,female,30.0,2012-08-27,no,37.0,yes,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
67731,2015-10-28,1054024,not-employee,ES,female,30.0,2012-08-27,no,38.0,yes,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0


In [6]:
g2 = dg.get_group(635245)
g2

,measurement_date,customer_id,employment_index,country_of_residence,gender,age,first_contract_date,is_new_customer,customer_for_months,is_primary_customer,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
77492,2015-11-28,635245,not-employee,ES,male,38.0,2006-08-19,yes,0.0,yes,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90958,2015-12-28,635245,not-employee,ES,male,38.0,2006-08-19,yes,1.0,no,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
g3 = dg.get_group(1477982)
g3

,measurement_date,customer_id,employment_index,country_of_residence,gender,age,first_contract_date,is_new_customer,customer_for_months,is_primary_customer,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
68850,2015-10-28,1477982,not-employee,ES,female,24.0,2015-10-13,yes,0.0,no,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
g4 = dg.get_group(1370908)
g4

,measurement_date,customer_id,employment_index,country_of_residence,gender,age,first_contract_date,is_new_customer,customer_for_months,is_primary_customer,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
2077,2015-01-28,1370908,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10830,2015-02-28,1370908,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17553,2015-03-28,1370908,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24090,2015-04-28,1370908,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
g5 = dg.get_group(1458089)
g5

,measurement_date,customer_id,employment_index,country_of_residence,gender,age,first_contract_date,is_new_customer,customer_for_months,is_primary_customer,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
65449,2015-09-28,1458089,not-employee,ES,male,47.0,2015-09-23,yes,0.0,no,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
g1.columns[23:]

Index([u'ind_ahor_fin_ult1', u'ind_aval_fin_ult1', u'ind_cco_fin_ult1',
       u'ind_cder_fin_ult1', u'ind_cno_fin_ult1', u'ind_ctju_fin_ult1',
       u'ind_ctma_fin_ult1', u'ind_ctop_fin_ult1', u'ind_ctpp_fin_ult1',
       u'ind_deco_fin_ult1', u'ind_deme_fin_ult1', u'ind_dela_fin_ult1',
       u'ind_ecue_fin_ult1', u'ind_fond_fin_ult1', u'ind_hip_fin_ult1',
       u'ind_plan_fin_ult1', u'ind_pres_fin_ult1', u'ind_reca_fin_ult1',
       u'ind_tjcr_fin_ult1', u'ind_valo_fin_ult1', u'ind_viv_fin_ult1',
       u'ind_nomina_ult1', u'ind_nom_pens_ult1', u'ind_recibo_ult1'],
      dtype='object')

In [139]:
def group_to_features(g, target):
    g = g.copy()
    
    d = {'customer_id': g.customer_id.max(),
         'measurement_month_last': g.measurement_date.max().month}
    
    d['customer_id'] = g.customer_id.max()
    d['measurement_month_last'] = g.measurement_date.max().month
    d['target'] = target
    
    last_id = g.index.tolist()[-1]
    
    for p in g.columns[23:]:
        g.set_value(last_id, p, np.nan)
    
    lca_features = [
        'employment_index',
        'country_of_residence',
        'is_new_customer',
        'is_primary_customer',
        'customer_type',
        'customer_relation_type',
        'country_residence_bank_same',
        'is_spouse_of_an_employee',
        'province_code',
        'is_active_customer',
        'household_income',
        'customer_segment']
    
    for p in g.columns[23:]:
        lca_features.append(p)
        d[p + '_sum'] = g[p].sum()
    
    for f in lca_features:
        try:
            d[f + '_last'] = g[f].dropna().tolist()[-1]
        except:
            d[f + '_last'] = np.nan
        d[f + '_changed'] = 'yes' if len(g[f].dropna().value_counts()) > 1 else 'no'
        d[f + '_changed_measurements_ago'] = np.nan
        if d[f + '_changed'] == 'yes':
            i = 2
            while len(g[f].dropna().tail(i).value_counts()) == 1:
                i += 1
            d[f + '_changed_measurements_ago'] = i - 1
    
    d['gender'] = g.gender.max()
    d['age'] = g.age.max()
    d['first_contract_month'] = g.first_contract_date.max().month
    d['age_at_first_contract'] = g.age.max() - (2016 - g.first_contract_date.max().year)
    d['diff_first_contract_month_and_last_month'] = (g.first_contract_date.max().month - g.measurement_date.max().month) % 12
    try:
        d['first_contract_months_ago'] = (g.measurement_date.max() - g.first_contract_date.max()).days / 30
    except:
        d['first_contract_months_ago'] = np.nan
    d['customer_for_months'] = g.customer_for_months.max()
    d['diff_first_customer_month_and_last_month'] = (g.measurement_date.max().month - g.customer_for_months.max()) % 12
    d['diff_first_customer_month_and_first_contract_month'] = (g.first_contract_date.max().month - g.customer_for_months.max()) % 12
    d['is_foreigner'] = g.is_foreigner.max()
    d['join_channel'] = g.join_channel.max()
    try:
        d['is_dead'] = g.is_dead.dropna().tolist()[-1]
    except:
        d['is_dead'] = np.nan
    d['household_income_diff'] = g['household_income'].tolist()[-1] - g['household_income'].tolist()[0]
    
    return d

l = []
l.append(group_to_features(g1, 'ind_ahor_fin_ult1'))
l.append(group_to_features(g2, 'ind_ahor_fin_ult1'))
l.append(group_to_features(g3, 'ind_ahor_fin_ult1'))
l.append(group_to_features(g4, 'ind_ahor_fin_ult1'))
l.append(group_to_features(g5, 'ind_ahor_fin_ult1'))
pd.DataFrame(l).T

,0,1,2,3,4
age,31,38,24,NaN,47
age_at_first_contract,27,28,23,NaN,46
country_of_residence_changed,no,no,no,no,no
country_of_residence_changed_measurements_ago,NaN,NaN,NaN,NaN,NaN
country_of_residence_last,ES,ES,ES,NaN,ES
country_residence_bank_same_changed,no,no,no,no,no
country_residence_bank_same_changed_measurements_ago,NaN,NaN,NaN,NaN,NaN
country_residence_bank_same_last,yes,yes,yes,NaN,yes
customer_for_months,46,1,0,NaN,0
customer_id,1054024,635245,1477982,1370908,1458089


In [119]:
d = pd.read_csv('tmp/clean_sample_100.csv',
        parse_dates=[
        'measurement_date',
        'first_contract_date'])
len(d)

1580

In [120]:
dg = d.groupby(['customer_id'])
len(dg)

100

In [15]:
l = []
for _, g in dg:
    l.append(group_to_features(g, 'ind_ahor_fin_ult1'))
pd.DataFrame(l).head(20).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
age,62,62,63,53,72,52,58,49,58,47,87,55,62,42,87,46,45,44,76,40
age_at_first_contract,42,43,44,35,55,35,41,34,42,31,71,40,47,27,77,32,31,30,62,26
country_of_residence_changed,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no
country_of_residence_changed_measurements_ago,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
country_of_residence_last,ES,ES,ES,ES,ES,ES,ES,ES,ES,ES,ES,ES,ES,ES,ES,ES,ES,ES,ES,ES
country_residence_bank_same_changed,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no
country_residence_bank_same_changed_measurements_ago,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
country_residence_bank_same_last,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes
customer_for_months,241,223,103,216,208,197,203,198,196,193,168,181,182,171,174,173,173,171,170,170
customer_id,39914,75349,80666,99205,122012,133882,139649,154758,160689,164341,201111,230998,237031,272728,310260,313766,317684,346499,350294,355558


In [233]:
d = pd.read_csv('tmp/clean_sample_1000.csv',
        parse_dates=[
        'measurement_date',
        'first_contract_date'])
dg = d.groupby(['customer_id'])
len(dg)

1000

In [234]:
cnt = {}
for target in g.columns[23:]:
    cnt[target] = 0
    cnt[target + '_cids'] = []
i = 1
for k, g in dg:
    if i % 1000 == 0:
        print i
    i += 1
    for target in g.columns[23:]:
        if len(g[target].dropna().value_counts()) > 1 and g[target].dropna().tolist()[0] == 0.0:
            cnt[target] = cnt[target] + 1
            cnt[target + '_cids'].append(k)
for k in g.columns[23:]:
    print k, cnt[k], cnt[k + '_cids'][:5]

0
100
200
300
400
500
600
700
800
900
ind_ahor_fin_ult1 0 []
ind_aval_fin_ult1 0 []
ind_cco_fin_ult1 46 [110373, 162238, 182141, 238709, 314843]
ind_cder_fin_ult1 0 []
ind_cno_fin_ult1 23 [120288, 201721, 263821, 396504, 407724]
ind_ctju_fin_ult1 0 []
ind_ctma_fin_ult1 16 [42055, 212764, 597925, 816319, 1393970]
ind_ctop_fin_ult1 0 []
ind_ctpp_fin_ult1 1 [396504]
ind_deco_fin_ult1 6 [610284, 1398985, 1431236, 1437556, 1447602]
ind_deme_fin_ult1 0 []
ind_dela_fin_ult1 9 [67144, 626574, 754092, 759455, 797044]
ind_ecue_fin_ult1 26 [110373, 120288, 178278, 212764, 597925]
ind_fond_fin_ult1 3 [182141, 450676, 774866]
ind_hip_fin_ult1 0 []
ind_plan_fin_ult1 0 []
ind_pres_fin_ult1 0 []
ind_reca_fin_ult1 5 [40733, 197100, 1262216, 1374070, 1377097]
ind_tjcr_fin_ult1 22 [23704, 122642, 160637, 220637, 234940]
ind_valo_fin_ult1 4 [42055, 141790, 450676, 776019]
ind_viv_fin_ult1 0 []
ind_nomina_ult1 27 [18773, 25379, 135107, 201721, 358765]
ind_nom_pens_ult1 27 [18773, 25379, 135107, 201721, 358

In [266]:
x = pd.read_csv('tmp/clean_sample_100.csv')
x.columns[23:]

Index([u'ind_ahor_fin_ult1', u'ind_aval_fin_ult1', u'ind_cco_fin_ult1',
       u'ind_cder_fin_ult1', u'ind_cno_fin_ult1', u'ind_ctju_fin_ult1',
       u'ind_ctma_fin_ult1', u'ind_ctop_fin_ult1', u'ind_ctpp_fin_ult1',
       u'ind_deco_fin_ult1', u'ind_deme_fin_ult1', u'ind_dela_fin_ult1',
       u'ind_ecue_fin_ult1', u'ind_fond_fin_ult1', u'ind_hip_fin_ult1',
       u'ind_plan_fin_ult1', u'ind_pres_fin_ult1', u'ind_reca_fin_ult1',
       u'ind_tjcr_fin_ult1', u'ind_valo_fin_ult1', u'ind_viv_fin_ult1',
       u'ind_nomina_ult1', u'ind_nom_pens_ult1', u'ind_recibo_ult1'],
      dtype='object')

In [235]:
target = 'ind_fond_fin_ult1'

d = pd.read_csv('tmp/clean_sample_10000.csv',
        parse_dates=[
        'measurement_date',
        'first_contract_date'])
dg = d.groupby(['customer_id'])

positive = []
negative = []

j = 1
for k, g in dg:
    if j % 1000 == 0:
        print j
    j += 1
    if len(g[target].dropna().value_counts()) > 1 and g[target].dropna().tolist()[0] == 0.0:
        i = 1
        while g[target].tolist()[i] != 1.0:
            i += 1
        positive.append(group_to_features(g.head(i + 1), 1))
    else:
        try:
            i = np.random.randint(1, len(g))
            negative.append(group_to_features(g.head(i), 0))
        except:
            pass
d2 = pd.DataFrame(positive + negative)
d2 = d2.iloc[np.random.permutation(len(d2))]
#print d2['target'].value_counts()


10000
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
0    9892
1      31
Name: target, dtype: int64


In [260]:
d3 = pd.read_csv('tmp/clean_sample_1000_classif_ind_ctma_fin_ult1.csv')
d3 = d3.iloc[np.random.permutation(len(d3))]
del d3['customer_id']
label = d3.target.copy()
del d3['target']
#d3.head()

In [261]:
cols_to_encode = []
for c in d3.columns:
    if d3[c].dtype == object:
        cols_to_encode.append(c)
#print cols_to_encode

In [262]:
d3 = pd.get_dummies(d3, columns=cols_to_encode)

In [263]:
tr = d3[:500].copy()
trl = label[:500].copy()
te = d3[500:].copy()
tel = label[500:].copy()

In [264]:
print trl.value_counts()
print tel.value_counts()

0    493
1      7
Name: target, dtype: int64
0    486
1      9
Name: target, dtype: int64


In [265]:
m = XGBClassifier(n_estimators=1000, max_depth=10)
m.fit(tr, trl)
p = m.predict(te)
print roc_auc_score(tel, p)
cmx = pd.DataFrame(confusion_matrix(tel, p))
cmx.index.name='actual'
cmx.columns.name='predicted'
print cmx

0.665637860082
predicted    0  1
actual           
0          485  1
1            6  3
